In [62]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score 
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score 
import optuna
import numpy as np 
import os
from sklearn import metrics
import matplotlib.pyplot as plt
from tqdm import tqdm

In [54]:
# 要跑 "ask" or "bid" 
STATE = "ask"
# Training and Validation  
# we split data to 10 parts
# TV1 means split data to training 1 part ,validation 9 parts
TV = 1
# 找參數要跑幾次
TRIALS = 1
# 是否要繼續train
PRE_TRAIN = True
# pre_train的次數
PRE_TRAIN_TIMES =50

In [47]:
# 估計值
def Score(m, x_train, y_train, x_test, y_test, train=True):
    # training 的
    if train:
        pred=m.predict(x_train)
        print('Train Result:\n')
        print(f"Accuracy Score: {accuracy_score(y_train, pred)*100:.2f}%")
        print(f"Precision Score: {precision_score(y_train, pred)*100:.2f}%")
        print(f"Recall Score: {recall_score(y_train, pred)*100:.2f}%")
        print(f"F1 score: {f1_score(y_train, pred)*100:.2f}%")
        print(f"Confusion Matrix:\n {confusion_matrix(y_train, pred)}")
    # testing 的
    elif train == False:
        pred=m.predict(x_test)
        print('Test Result:\n')
        print(f"Accuracy Score: {accuracy_score(y_test, pred)*100:.2f}%")
        print(f"Precision Score: {precision_score(y_test, pred)*100:.2f}%")
        print(f"Recall Score: {recall_score(y_test, pred)*100:.2f}%")
        print(f"F1 score: {f1_score(y_test, pred)*100:.2f}%")
        print(f"Confusion Matrix:\n {confusion_matrix(y_test, pred)}")
        

In [48]:
# 讀data
state = "ask"
FILE_PATH = f"/kaggle/input/hfs-dataset/stock_dataset_with_label/stock_dataset_with_label/2330/{STATE}/*"

files = glob.glob(FILE_PATH)

data=[]
data = pd.DataFrame(data)

for f in files:
    
    d = pd.read_csv(f, index_col=None)
    d = d.drop(columns=["matchPri",'bidPri1','bidPri2','bidPri3','bidPri4','bidPri5','askPri1','askPri2','askPri3','askPri4','askPri5',"openPri"])
    data = pd.concat([data,d]) 
    

print(data.shape)

(527175, 12)


In [20]:
# the length of total data 
length = len(data)
# the length of training data 
train_length = int(length * TV *0.1)
# split training data
train_data = data[:train_length]
# train input data 
X_train = train_data.drop(columns=['label'])
# train label 
y_train = train_data['label'].to_numpy()

# split testing data
test_data = data[train_length:]
# tets input data 
X_test = test_data.drop(columns=['label'])
# test label
y_test = test_data['label'].to_numpy()



In [51]:
print(f"Train : {len(X_train)} {len(y_train)}")
print(f"Test  : {len(X_test)} {len(y_test)}")

Train : 52717 52717
Test  : 474458 474458


In [50]:
# 計算training 的 label 比例分配
change = 0
other = 0
for i in y_train:
    
    if i == 0:
        other+=1
    else:
        change+=1

print(change,other)
print('change: {:%}'.format(change/(change+other)))
print('other : {:%}'.format(other/(change+other)))

8472 44245
change: 16.070717%
other : 83.929283%


In [49]:
best_recall = -1

In [52]:
# run weight 
def Objective(trial):

    global best_recall
    # 配置要選的權重和range
    # 'scale_pos_weight':other/down,這個一定要加 ，他是處理imbalance的 
    param = {
        'max_depth': trial.suggest_int('max_depth', 2, 15),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0,log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50,500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0,log=True),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0,log=True),
        'scale_pos_weight':other/change,
        'reg_alpha':  trial.suggest_float('reg_lambda', 1e-8, 1.0,log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0,log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.01, 1.0),
        # CPU thread 
        'nthread':16
    }

    # 訓練
    model = xgb.XGBClassifier(**param)  
    
    model.fit(X_train, y_train,verbose=False)
    # 預測
    X_pred = model.predict(X_train)
    # 評估分數
    recall = round(f1_score(y_train, X_pred),2)
    if recall > best_recall:
        # 存model weight
        model.save_model(f'./xgb_best_{STATE}{recall}.model')

        if os.path.isfile(f'./xgb_best_{STATE}{best_recall}.model'):
            os.remove(f'./xgb_best_{STATE}{best_recall}.model')
        
        best_recall = recall
        
    return recall

In [55]:
# 找參數的套件 ㄝㄝ, 很佔資源
# direction ="maximize" ,代表作為評估的值要找最大值
study = optuna.create_study(direction='maximize')

# pre-train 用

# xgb_model = xgb.XGBClassifier()
# xgb_model.load_model(model_path)
# study.enqueue_trial(xgb_model.get_params())

# n_trials 要跑幾次
study.optimize(Objective, n_trials = TRIALS,show_progress_bar=True)

[I 2023-03-20 16:39:22,896] A new study created in memory with name: no-name-e109d22c-c7f7-421b-b13c-13acf0b1466f
/opt/conda/lib/python3.7/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/1 [00:00<?, ?it/s]

[I 2023-03-20 16:39:36,680] Trial 0 finished with value: 0.33 and parameters: {'max_depth': 11, 'subsample': 0.030530972049734588, 'n_estimators': 318, 'learning_rate': 0.048517179194020724, 'gamma': 0.014790947908933075, 'reg_lambda': 1.1915447304249974e-05, 'min_child_weight': 14, 'colsample_bytree': 0.912857937589227}. Best is trial 0 with value: 0.33.


In [57]:
# 輸出找到的最佳參數
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 1
Best trial:
  Value: 0.33
  Params: 
    max_depth: 11
    subsample: 0.030530972049734588
    n_estimators: 318
    learning_rate: 0.048517179194020724
    gamma: 0.014790947908933075
    reg_lambda: 1.1915447304249974e-05
    min_child_weight: 14
    colsample_bytree: 0.912857937589227


In [59]:
# 看一下效果
model = xgb.XGBClassifier()

# 放best model
model_path = f'./xgb_best_{state}{round(study.best_value,2)}.model'

model.load_model(model_path)
# training eval
Score(model,X_train, y_train, X_test,y_test)
# testing eval
Score(model,X_train, y_train, X_test,y_test,train=False)

print('特徵重要程度: ',model.feature_importances_)

Train Result:

Accuracy Score: 67.76%
Precision Score: 24.63%
Recall Score: 48.83%
F1 score: 32.74%
Confusion Matrix:
 [[31585 12660]
 [ 4335  4137]]
Test Result:

Accuracy Score: 59.05%
Precision Score: 17.80%
Recall Score: 39.12%
F1 score: 24.47%
Confusion Matrix:
 [[248691 145319]
 [ 48976  31472]]
特徵重要程度:  [0.09926887 0.08794466 0.08915552 0.09162249 0.08911469 0.09227598
 0.08960105 0.08945271 0.09056973 0.08960213 0.0913922 ]


In [ ]:
# 這邊就不斷調要train 幾次 ,看recall score 
# **trial.params : 前面找出來的參數

if PRE_TRAIN:
    xgb_model = xgb.XGBClassifier(**trial.params)
#     pre-train用
#     xgb_model = xgb.XGBClassifier()
#     xgb_model.load_model(model_path)
    
    loop = tqdm(range(PRE_TRAIN_TIMES))

    for i in loop:
        # xgb_model 讀檔的
        xgb_model.fit(X_train, y_train,verbose=1,xgb_model=model_path)
        # 存檔
        xgb_model.save_model(model_path)


    Score(xgb_model,X_train, y_train, X_test,y_test)
    Score(xgb_model,X_train, y_train, X_test,y_test,train=False)